#**유통 판매량 예측 및 재고 최적화**
## **단계2 : 모델링을 위한 데이터 전처리**

# **0.미션**

* 단계1 상품별 데이터셋 만들기
    * 가설로 도출된 변수를 반영한 데이터셋을 상품별로 생성
* 단계2 기초 모델 생성
    * base line model : dense layer만 이용해서 기초 모델 생성
    * LSTM, CNN 알고리즘을 이용해서 기초 모델 생성

* 대상 상품(핵심 상품)

| Product_ID|Product_Code|SubCategory|Category|LeadTime|Price|
|----|----|----|----|----|----|
|3|DB001|Beverage|Drink|2|8|
|12|GA001|Milk|Food|3|6|
|42|FM001|Agricultural products|Grocery|3|5|


# **1.환경설정**

* 세부 요구사항
    * 구글콜랩
        * 구글 드라이브 바로 밑에 project 폴더를 만들고,
        * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 로딩
        * 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
        * 필요하다고 판단되는 라이브러리를 추가하세요.


### **(1) 경로 설정**

* 구글 드라이브 연결

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# path = '/content/drive/MyDrive/project/'

### **(2) 라이브러리 설치 및 불러오기**

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

from keras.models import Sequential
from keras.backend import clear_session
from keras.layers import Dense, LSTM, Conv1D, Flatten, Input
from keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [2]:
# 학습곡선 함수
def dl_history_plot(history):
    plt.plot(history['loss'], label='train_err', marker = '.')
    plt.plot(history['val_loss'], label='val_err', marker = '.')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid()
    plt.show()

In [3]:
# 예측 결과 시각화
def plot_model_result(y_train, y_val, pred) :

    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_val.index = range(len(y_train), len(y_train) + len(y_val))

    pred = pd.Series(pred.reshape(-1,), index = y_val.index)

    # 전체 시각화
    plt.figure(figsize = (20,12))
    plt.subplot(2,1,1)
    plt.plot(y_train, label = 'train')
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.subplot(2,1,2)
    plt.plot(y_val, label = 'val')
    plt.plot(pred, label = 'pred')
    plt.legend()
    plt.grid()

    plt.show()

### **(3) 데이터 불러오기**

In [4]:
import os
import zipfile
from tqdm import tqdm
import gdown

def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

def dataset_extract(file_name) :

    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[:-4]}/') :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[:-4]}/')

In [30]:
file_id = '16680amomXMteqrTs7SgvlUrNDf7TS-rU'
download_file(file_id, 'train.zip')
file_id = '1VI2j8lCkGwV0qmB36jWT-1P0gYnKfCJh'
download_file(file_id, 'test.zip')
file_id = '1G7NJ2tjotwcI39nHt6SmxgkqYPi9Cl0k'
download_file(file_id, 'total_df.csv')


dataset_extract('train.zip')
dataset_extract('test.zip')

train.zip 파일이 이미 존재합니다.
test.zip 파일이 이미 존재합니다.


Downloading...
From: https://drive.google.com/uc?id=1G7NJ2tjotwcI39nHt6SmxgkqYPi9Cl0k
To: /content/total_df.csv
100%|██████████| 413k/413k [00:00<00:00, 61.6MB/s]

데이터셋 폴더가 이미 존재합니다.
데이터셋 폴더가 이미 존재합니다.


In [6]:
path = './'

oil_price = pd.read_csv(path + 'train/oil_price_train.csv')
orders = pd.read_csv(path + 'train/orders_train.csv')
sales = pd.read_csv(path + 'train/sales_train.csv')
products = pd.read_csv(path + 'train/products.csv')
stores = pd.read_csv(path + 'train/stores.csv')

sales['Date'] = pd.to_datetime(sales['Date'] )
oil_price['Date'] = pd.to_datetime(oil_price['Date'] )
orders['Date'] = pd.to_datetime(orders['Date'] )

In [26]:
orders.head()

,Date,Store_ID,CustomerCount
0,2014-01-01,25,840
1,2014-01-01,36,487
2,2014-01-02,1,1875
3,2014-01-02,2,2122
4,2014-01-02,3,3350


# **2.데이터셋 구성하기**

* 세부 요구사항
    * 상품코드 별로 각각 데이터셋을 구성해야 합니다.
    * 도출된 변수들을 포함하는 데이터셋을 구성합니다.


In [271]:
# sales와 oil_price merge
df = pd.merge(sales.loc[(sales['Store_ID']==44)&(sales['Product_ID']==12),:],oil_price,
              left_on='Date', right_on='Date', how='left')
# orders merge
df = pd.merge(df, orders.loc[orders['Store_ID']==44, ['Date', 'CustomerCount']],
              left_on='Date', right_on='Date', how='left')
df

,Date,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount
0,2014-01-01,44,0.0,12,NaN,NaN
1,2014-01-02,44,9647.0,12,95.14,4422.0
2,2014-01-03,44,8879.0,12,93.66,4167.0
3,2014-01-04,44,14188.0,12,NaN,5341.0
4,2014-01-05,44,14490.0,12,NaN,5123.0
...,...,...,...,...,...,...
1147,2017-02-24,44,10630.0,12,53.99,4549.0
1148,2017-02-25,44,12978.0,12,NaN,4619.0
1149,2017-02-26,44,10899.0,12,NaN,3877.0
1150,2017-02-27,44,10618.0,12,54.04,3961.0


In [327]:
df = pd.read_csv('./total_df.csv')
df['Date'] = pd.to_datetime(df['Date'])
# oil_price 선형보간 2014/01/01 = 98.17

df.loc[(df['Date']>'2016-01-01')&(df['Date']<'2016-01-10'),'CustomerCount'] = df.loc[(df['Date']>'2016-01-01')&(df['Date']<'2016-01-10'),'CustomerCount'].interpolate()
df['CustomerCount'] = df['CustomerCount'].fillna(0)
df

,Date,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount,Product_Code,SubCategory,Category,LeadTime,Price,City,State,Store_Type,year,month,day,weekday,season,holiday
0,2014-01-01,44,0.0,3,98.17,0.0,DB001,Beverage,Drink,2,8,Saint Paul,\tMinnesota,1,2014,1,1,Wednesday,Winter,False
1,2014-01-01,44,0.0,12,98.17,0.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,1,Wednesday,Winter,False
2,2014-01-01,44,51.0,42,98.17,0.0,FM001,Agricultural products,Grocery,2,5,Saint Paul,\tMinnesota,1,2014,1,1,Wednesday,Winter,False
3,2014-01-02,44,9853.0,3,95.14,4422.0,DB001,Beverage,Drink,2,8,Saint Paul,\tMinnesota,1,2014,1,2,Thursday,Winter,False
4,2014-01-02,44,9647.0,12,95.14,4422.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,2,Thursday,Winter,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,2017-02-27,44,10618.0,12,54.04,3961.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,27,Monday,Winter,False
3455,2017-02-27,44,78.0,42,54.04,3961.0,FM001,Agricultural products,Grocery,2,5,Saint Paul,\tMinnesota,1,2017,2,27,Monday,Winter,False
3456,2017-02-28,44,14374.0,3,54.00,4589.0,DB001,Beverage,Drink,2,8,Saint Paul,\tMinnesota,1,2017,2,28,Tuesday,Winter,False
3457,2017-02-28,44,14400.0,12,54.00,4589.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,28,Tuesday,Winter,False


## **(1) 상품 : 3 - Beverage**

## **(2) 상품 : 12 - Milk**

##### 데이터 준비

In [328]:
df_12 = df.loc[(df['Store_ID']==44)&(df['Product_ID']==12),:]
df_12.reset_index(inplace=True,drop=True)
df_12

,Date,Store_ID,Qty,Product_ID,WTI_Price,CustomerCount,Product_Code,SubCategory,Category,LeadTime,Price,City,State,Store_Type,year,month,day,weekday,season,holiday
0,2014-01-01,44,0.0,12,98.17,0.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,1,Wednesday,Winter,False
1,2014-01-02,44,9647.0,12,95.14,4422.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,2,Thursday,Winter,False
2,2014-01-03,44,8879.0,12,93.66,4167.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,3,Friday,Winter,False
3,2014-01-04,44,14188.0,12,93.66,5341.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,4,Saturday,Winter,False
4,2014-01-05,44,14490.0,12,93.66,5123.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2014,1,5,Sunday,Winter,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2017-02-24,44,10630.0,12,53.99,4549.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,24,Friday,Winter,False
1148,2017-02-25,44,12978.0,12,53.99,4619.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,25,Saturday,Winter,False
1149,2017-02-26,44,10899.0,12,53.99,3877.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,26,Sunday,Winter,False
1150,2017-02-27,44,10618.0,12,54.04,3961.0,GA001,Milk,Food,2,6,Saint Paul,\tMinnesota,1,2017,2,27,Monday,Winter,False


In [329]:
christmas = {'Date':['2014-12-25','2015-12-25','2016-12-25'],
             'Qty':[0,0,0],
             'CustomerCount':[0,0,0],
             'holiday': [True,True,True]}
christmas = pd.DataFrame(christmas)
christmas['Date'] = pd.to_datetime(christmas['Date'])

df_12 = pd.concat([df_12,christmas],axis=0)
# df_12['WTI_Price']
df_12 = df_12.sort_values('Date')
df_12['WTI_Price'] = df_12['WTI_Price'].interpolate()
df_12 = df_12.fillna(method='ffill')
df_12['year'] = df_12['Date'].dt.year
df_12['month'] = df_12['Date'].dt.month
df_12['day'] = df_12['Date'].dt.day
df_12['weekday'] = df_12['Date'].dt.weekday
df_12.drop(['Product_Code','SubCategory','Category','LeadTime','City','State','Store_Type','Price','Product_ID','Store_ID'], axis=1,inplace=True)
df_12.reset_index(inplace=True, drop=True)
df_12

,Date,Qty,WTI_Price,CustomerCount,year,month,day,weekday,season,holiday
0,2014-01-01,0.0,98.17,0.0,2014,1,1,2,Winter,False
1,2014-01-02,9647.0,95.14,4422.0,2014,1,2,3,Winter,False
2,2014-01-03,8879.0,93.66,4167.0,2014,1,3,4,Winter,False
3,2014-01-04,14188.0,93.66,5341.0,2014,1,4,5,Winter,False
4,2014-01-05,14490.0,93.66,5123.0,2014,1,5,6,Winter,False
...,...,...,...,...,...,...,...,...,...,...
1150,2017-02-24,10630.0,53.99,4549.0,2017,2,24,4,Winter,False
1151,2017-02-25,12978.0,53.99,4619.0,2017,2,25,5,Winter,False
1152,2017-02-26,10899.0,53.99,3877.0,2017,2,26,6,Winter,False
1153,2017-02-27,10618.0,54.04,3961.0,2017,2,27,0,Winter,False


In [330]:
# 시계열 분석
decomp = sm.tsa.seasonal_decompose(df_12['Qty'], model = 'additive', period = 7)
result = pd.DataFrame({'observed':decomp.observed, 'trend':decomp.trend, 'seasonal':decomp.seasonal, 'residual':decomp.resid})
display(result)

,observed,trend,seasonal,residual
0,0.0,NaN,-634.394710,NaN
1,9647.0,NaN,-2655.647424,NaN
2,8879.0,NaN,-1148.132777,NaN
3,14188.0,8848.857143,2859.619599,2479.523258
4,14490.0,10089.285714,3658.964784,741.749501
...,...,...,...,...
1150,10630.0,10010.428571,-1148.132777,1767.704205
1151,12978.0,11014.142857,2859.619599,-895.762456
1152,10899.0,NaN,3658.964784,NaN
1153,10618.0,NaN,-724.764944,NaN


In [331]:
df_12 = pd.merge(df_12,result,left_index=True, right_index=True, how='left')
df_12

,Date,Qty,WTI_Price,CustomerCount,year,month,day,weekday,season,holiday,observed,trend,seasonal,residual
0,2014-01-01,0.0,98.17,0.0,2014,1,1,2,Winter,False,0.0,NaN,-634.394710,NaN
1,2014-01-02,9647.0,95.14,4422.0,2014,1,2,3,Winter,False,9647.0,NaN,-2655.647424,NaN
2,2014-01-03,8879.0,93.66,4167.0,2014,1,3,4,Winter,False,8879.0,NaN,-1148.132777,NaN
3,2014-01-04,14188.0,93.66,5341.0,2014,1,4,5,Winter,False,14188.0,8848.857143,2859.619599,2479.523258
4,2014-01-05,14490.0,93.66,5123.0,2014,1,5,6,Winter,False,14490.0,10089.285714,3658.964784,741.749501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1150,2017-02-24,10630.0,53.99,4549.0,2017,2,24,4,Winter,False,10630.0,10010.428571,-1148.132777,1767.704205
1151,2017-02-25,12978.0,53.99,4619.0,2017,2,25,5,Winter,False,12978.0,11014.142857,2859.619599,-895.762456
1152,2017-02-26,10899.0,53.99,3877.0,2017,2,26,6,Winter,False,10899.0,NaN,3658.964784,NaN
1153,2017-02-27,10618.0,54.04,3961.0,2017,2,27,0,Winter,False,10618.0,NaN,-724.764944,NaN


In [332]:
#2일뒤의 값으로 해야하는 컬럼 weekday season holiday, target
df_12['target'] = df_12['Qty'].shift(-2)

df_12['year'] = df_12['year'].shift(-2)
df_12['month'] = df_12['month'].shift(-2)
df_12['day'] = df_12['day'].shift(-2)
df_12['weekday'] = df_12['weekday'].shift(-2)
df_12['season'] = df_12['season'].shift(-2)
# df_12['holiday'] = df_12['holiday'].shift(-2)
df_12.dropna(inplace=True)
df_12

,Date,Qty,WTI_Price,CustomerCount,year,month,day,weekday,season,holiday,observed,trend,seasonal,residual,target
3,2014-01-04,14188.0,93.66,5341.0,2014.0,1.0,6.0,0.0,Winter,False,14188.0,8848.857143,2859.619599,2479.523258,7614.0
4,2014-01-05,14490.0,93.66,5123.0,2014.0,1.0,7.0,1.0,Winter,False,14490.0,10089.285714,3658.964784,741.749501,7124.0
5,2014-01-06,7614.0,93.12,3917.0,2014.0,1.0,8.0,2.0,Winter,False,7614.0,9672.714286,-724.764944,-1333.949342,8683.0
6,2014-01-07,7124.0,93.31,3663.0,2014.0,1.0,9.0,3.0,Winter,False,7124.0,9403.142857,-1355.644528,-923.498329,6731.0
7,2014-01-08,8683.0,91.90,4254.0,2014.0,1.0,10.0,4.0,Winter,False,8683.0,8979.714286,-634.394710,337.680425,6992.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2017-02-21,7374.0,54.02,3371.0,2017.0,2.0,23.0,3.0,Winter,False,7374.0,10199.714286,-1355.644528,-1470.069757,8643.0
1148,2017-02-22,8931.0,53.61,3834.0,2017.0,2.0,24.0,4.0,Winter,False,8931.0,9654.714286,-634.394710,-89.319575,10630.0
1149,2017-02-23,8643.0,54.48,3955.0,2017.0,2.0,25.0,5.0,Winter,False,8643.0,9528.714286,-2655.647424,1769.933138,12978.0
1150,2017-02-24,10630.0,53.99,4549.0,2017.0,2.0,26.0,6.0,Winter,False,10630.0,10010.428571,-1148.132777,1767.704205,10899.0


In [333]:
df_12['Date']

3      2014-01-04
4      2014-01-05
5      2014-01-06
6      2014-01-07
7      2014-01-08
          ...    
1147   2017-02-21
1148   2017-02-22
1149   2017-02-23
1150   2017-02-24
1151   2017-02-25
Name: Date, Length: 1149, dtype: datetime64[ns]

In [334]:
X = df_12.drop(['Date','target','year', 'day','observed'],axis=1)
y = df_12.loc[:,'target']

In [335]:
X = pd.get_dummies(X,columns=['weekday','season'],drop_first=True)

In [336]:
X

,Qty,WTI_Price,CustomerCount,month,holiday,trend,seasonal,residual,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,weekday_6.0,season_Spring,season_Summer,season_Winter
3,14188.0,93.66,5341.0,1.0,False,8848.857143,2859.619599,2479.523258,False,False,False,False,False,False,False,False,True
4,14490.0,93.66,5123.0,1.0,False,10089.285714,3658.964784,741.749501,True,False,False,False,False,False,False,False,True
5,7614.0,93.12,3917.0,1.0,False,9672.714286,-724.764944,-1333.949342,False,True,False,False,False,False,False,False,True
6,7124.0,93.31,3663.0,1.0,False,9403.142857,-1355.644528,-923.498329,False,False,True,False,False,False,False,False,True
7,8683.0,91.90,4254.0,1.0,False,8979.714286,-634.394710,337.680425,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,7374.0,54.02,3371.0,2.0,False,10199.714286,-1355.644528,-1470.069757,False,False,True,False,False,False,False,False,True
1148,8931.0,53.61,3834.0,2.0,False,9654.714286,-634.394710,-89.319575,False,False,False,True,False,False,False,False,True
1149,8643.0,54.48,3955.0,2.0,False,9528.714286,-2655.647424,1769.933138,False,False,False,False,True,False,False,False,True
1150,10630.0,53.99,4549.0,2.0,False,10010.428571,-1148.132777,1767.704205,False,False,False,False,False,True,False,False,True


In [337]:
from sklearn.preprocessing import MinMaxScaler
cols = X.columns
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X =pd.DataFrame(X)
X.columns = cols

y_max, y_min = y.max(), y.min()
y = (y-y_min) / (y_max-y_min)

In [338]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X,y,test_size=60,shuffle=False)

In [339]:
X

,Qty,WTI_Price,CustomerCount,month,holiday,trend,seasonal,residual,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,weekday_6.0,season_Spring,season_Summer,season_Winter
0,0.382364,0.825220,0.638952,0.000000,0.0,0.114444,0.873413,0.499524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.390503,0.825220,0.612872,0.000000,0.0,0.204307,1.000000,0.452486,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.205196,0.818615,0.468597,0.000000,0.0,0.174129,0.305780,0.396301,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.191991,0.820939,0.438210,0.000000,0.0,0.154599,0.205872,0.407411,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.234005,0.803694,0.508913,0.000000,0.0,0.123924,0.320091,0.441548,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144,0.198728,0.340386,0.403278,0.090909,0.0,0.212308,0.205872,0.392617,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1145,0.240689,0.335372,0.458667,0.090909,0.0,0.172825,0.320091,0.429990,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1146,0.232927,0.346013,0.473143,0.090909,0.0,0.163696,0.000000,0.480316,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1147,0.286477,0.340020,0.544204,0.090909,0.0,0.198595,0.238734,0.480256,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## **(3) 상품 : 42 - Agricultural products**

# **3.Baseline Model**

* **세부 요구사항**
    * 데이터 준비
        * 스케일링
            * 교과서 적으로는 데이터 분할 이후 스케일링하는게 맞지만, 데이터 건수가 충분히 크다면, 스케일링을 먼저 하기도 합니다.
            * 여기서는 먼저 스케일링을 한 후 데이터 분할을 합니다.
        * 데이터 분할
            * 검증셋 구성 : 학습용 데이터에서 최근 60일을 검증셋으로 사용.

    * 모델링
        * Dense Layer 1개(output layer)만으로 모델을 생성합니다.
        * 모델의 검증 성능을 평가하고 기록합니다.
        * 평가 지표 : RMSE, MAE, MAPE, R2 Score


In [320]:
x_train.astype(float).dtypes

AttributeError: 'numpy.ndarray' object has no attribute 'dtypes'

In [321]:
x_train

array([[0.38236404, 0.82522016, 0.63895203, ..., 0.        , 0.        ,
        1.        ],
       [0.39050288, 0.82522016, 0.61287235, ..., 0.        , 0.        ,
        1.        ],
       [0.20519593, 0.81861546, 0.46859672, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.31580235, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.38621786, 0.31580235, 0.67771265, ..., 0.        , 0.        ,
        1.        ],
       [0.25437935, 0.32570939, 0.55927743, ..., 0.        , 0.        ,
        1.        ]])

In [322]:
d_model = Sequential()
d_model.add(Input(shape=(x_train.shape[1],)))
d_model.add(Dense(128,activation='relu'))
d_model.add(Dense(1))

d_model.summary()

d_model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mae','mape'])

d_model.fit(x_train.astype(float),y_train, validation_data=(x_val.astype(float),y_val), epochs=300)

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_70 (Dense)            (None, 128)               2304      
                                                                 
 dense_71 (Dense)            (None, 1)                 129       
                                                                 
Total params: 2433 (9.50 KB)
Trainable params: 2433 (9.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/300
35/35 [==============================] - 1s 8ms/step - loss: 0.0490 - mae: 0.1627 - mape: 1496065.7500 - val_loss: 0.0088 - val_mae: 0.0701 - val_mape: 4641478.0000
Epoch 2/300
35/35 [==============================] - 0s 3ms/step - loss: 0.0055 - mae: 0.0511 - mape: 1347895.6250 - val_loss: 0.0067 - val_mae: 0.0556 - val_mape: 4860968.5000
Epoch 3/300
35/35 [==============================] - 0s 3ms/ste

In [240]:
from sklearn.metrics import *
pred = d_model.predict(x_val)
pred = pred*(y_max-y_min) + y_min
y_val_decode = y_val*(y_max-y_min) + y_min

print(mean_absolute_error(y_val_decode,pred))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

## **(1) 상품 : 3 - Beverage**

### **1) 데이터 준비**

### **2) 모델링**

In [ ]:
c_model

## **(2) 상품 : 12 - Milk**

### **1) 데이터 준비**

### **2) 모델링**

## **(3) 상품 : 42 - Agricultural products**

### **1) 데이터 준비**

### **2) 모델링**

# **4.LSTM, CNN 초기 모델**

* **세부 요구사항**
    * 3차원 데이터 구조 만들기
        * timestep 수는 적절한 간격으로 한가지 경우만 지정을 합니다.
    * LSTM, CNN 기반 초기 모델 생성
        * 성능 보다는 코드 틀을 작성하는데 집중합시다.
        * 노드 혹은 필터 수와 크기는 초기값으로 적절하게 지정해 봅시다.

In [340]:
def temporalize(x, y, timesteps):
    output_X = []
    output_y = []
    for i in range(len(x) - timesteps + 1):
        output_X.append(x.iloc[i:i+timesteps])
        output_y.append(y.iloc[i+timesteps-1])
    return np.array(output_X), np.array(output_y)

In [348]:
t_X, t_y = temporalize(X, y, 7)

t_x_train, t_x_val, t_y_train, t_y_val = train_test_split(t_X,t_y,test_size=60, shuffle=False)

## **(1) LSTM 초기 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [245]:
t_X.shape, t_y.shape

((1143, 7, 27), (1143,))

* **모델링**

In [349]:
l_model = Sequential()
l_model.add(Input(shape=(7,17)))
l_model.add(LSTM(32,return_sequences=True))
l_model.add(LSTM(8,return_sequences=True))
l_model.add(LSTM(4,return_sequences=True))
l_model.add(LSTM(1,return_sequences=True))
l_model.add(Flatten())
l_model.add(Dense(8, activation='relu'))
l_model.add(Dense(1))

l_model.summary()

l_model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mae'])

l_model.fit(t_x_train.astype(float),t_y_train, validation_data=(t_x_val.astype(float),t_y_val), epochs=300)

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 7, 32)             6400      
                                                                 
 lstm_21 (LSTM)              (None, 7, 8)              1312      
                                                                 
 lstm_22 (LSTM)              (None, 7, 4)              208       
                                                                 
 lstm_23 (LSTM)              (None, 7, 1)              24        
                                                                 
 flatten_9 (Flatten)         (None, 7)                 0         
                                                                 
 dense_76 (Dense)            (None, 8)                 64        
                                                                 
 dense_77 (Dense)            (None, 1)               

### **2) 상품 : 12 - Milk**

* **데이터 준비**

* **모델링**

### **3) 상품 : 42 - Agricultural products**

* **데이터 준비**

* **모델링**

## **(2) CNN 초기 모델링**

### **1) 상품 : 3 - Beverage**

* **데이터 준비**

In [342]:
t_x_train.shape

(1083, 7, 17)

In [351]:
c_model = Sequential()
c_model.add(Input(shape=(7,17)))
c_model.add(Conv1D(32, 5, activation='relu', padding = 'same'))
c_model.add(Flatten())
c_model.add(Dense(1))

c_model.summary()

c_model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mae'])

c_model.fit(t_x_train,t_y_train, validation_data=(t_x_val,t_y_val), epochs=300)

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 7, 32)             2752      
                                                                 
 flatten_11 (Flatten)        (None, 224)               0         
                                                                 
 dense_79 (Dense)            (None, 1)                 225       
                                                                 
Total params: 2977 (11.63 KB)
Trainable params: 2977 (11.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/300
34/34 [==============================] - 2s 12ms/step - loss: 0.0184 - mae: 0.0949 - val_loss: 0.0129 - val_mae: 0.0752
Epoch 2/300
34/34 [==============================] - 0s 4ms/step - loss: 0.0052 - mae: 0.0480 - val_loss: 0.0103 - val_mae: 0.0648
Epoch 3/300
34/34 [=

* **모델링**

### **2) 상품 : 12 - Milk**

* **데이터 준비**

* **모델링**

### **3) 상품 : 42 - Agricultural products**

* **데이터 준비**

* **모델링**

# **5.데이터 저장**

* **세부 요구사항**
    * 상품코드별 데이터셋을 저장하시오.
    * joblib.dump를 이용하시오.
    * 저장할 파일의 확장자는 보통 .pkl 입니다.